# Reverse engineer the rank algorithm

Adapted from [calculateRank.js][1]

[1]: https://github.com/anuraghazra/github-readme-stats/blob/f83080601e87dfe76b003cd08c675cbb2f6204e9/src/calculateRank.js#L2

### Define enums and stats class

In [ ]:
offset = Dict([("commits", 1.65),
               ("contribs", 1.65),
               ("issues", 1),
               ("stars", 0.75),
               ("prs", 0.5),
               ("followers", 0.45),
               ("repos", 1)])

In [ ]:
@enum Rank begin
    s = 1
    a1 = 25
    a2 = 45
    a3 = 60
    b = 100
end

In [ ]:
function normalcdf(μ, σ, ∑)
    z = (∑ - μ) / sqrt(2 * σ * σ) 
    t = 1 / (1 + 0.3275911 * abs(z))
    w = (0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429)
    erf = 1 - ((((w[5] * t + w[4]) * t + w[3]) * t + w[2]) * t + w[1]) * t * exp(-z * z)
    sign = z < 0 ? -1 : 1
    (1 + sign * erf) / 2
end

### Define compute functions

In [ ]:
function compute_score(stats)
    @assert keys(offset) == keys(stats)
    score = 0
    for key in keys(offset)
        score += offset[key] * stats[key]
    end
    score / 100
end

In [ ]:
function compute_normalized_score(score)
    total_rank = sum(Integer, [s, a1, a2, a3, b])
    total_offset = sum(values(offset))
    normalized = normalcdf(score, total_rank, total_offset)
    normalized * 100
end

compute_normalized_score(0)

In [ ]:
function compute_rank(normalized_score)
    if normalized_score < Integer(s)
        level = "S+"
    elseif normalized_score < Integer(a1)
        level = "S"
    elseif normalized_score < Integer(a2)
        level = "A++"
    elseif normalized_score < Integer(a3)
        level = "A+"
    elseif normalized_score < Integer(b)
        level = "B+"
    else
        level = "B"
    end
    level
end

compute_normalized_score(0)

### Sanity check with my GitHub stats

In [ ]:
my_stats = Dict([("repos", 33),
                 ("commits", 880),
                 ("contribs", 7),
                 ("followers", 5),
                 ("prs", 30),
                 ("issues", 41),
                 ("stars", 1)])
my_score = compute_score(my_stats)
my_normalized_score = compute_normalized_score(my_score)
my_rank = compute_rank(my_normalized_score)
println("score = $my_score rank = $my_rank")

### Everyone gets an A+!

In [ ]:
zero_normalized_score = compute_normalized_score(0)
zero_rank = compute_rank(zero_normalized_score)
zero_rank

### Find the rank distribution
there's probably an O(1) way to do this...

In [ ]:
def binary_search(function, target, lower=0, upper=1e4):
    mid = -1
    while lower + 1 < upper:
        mid = (lower + upper) // 2
        if function(mid) >= target:
            lower = mid
        else:
            upper = mid
    return mid


find_distribution = partial(binary_search, function=compute_normalized_score)

In [ ]:
reachables = (Rank.A2, Rank.DOUBLE_A, Rank.S)
names = ('A++', 'S', 'S+')
thresholds = {}
for reachable, name in zip(reachables, names):
    result = find_distribution(target=reachable)
    print(f"Score needed to reach {name}: {result}")
    thresholds[name] = result

### How many commits until I reach the next rank?

In [ ]:
def calculate_commits(current, next):
    assert current < next
    gap = (next - current) * 100
    commits = round(gap / Offset.COMMITS)
    per_day = round(commits / 365, 1)
    print(f"You need {commits} commits, or around "
          f"{per_day} commits a day for the next 365 days.")

In [ ]:
calculate_commits(my_score, thresholds['A++'])